In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

In [2]:
path_rawdata    = 'F://TFG//datasets/raw_datasets//'
path_train      = 'F://TFG//datasets//data_train//'
path_graphs     = 'F://TFG//graphs//'
path            = 'F:/TFG/datasets/nature-dataset/'

## Wyscout processing

In [3]:
data        = pd.read_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';',index_col='wyId')
teams       = pd.read_csv(path_rawdata+'teams.csv',sep=';')
raw_Data    = pd.read_json(path_rawdata+'RAW_partidos.json')

In [4]:
data = (pd.merge(data.reset_index(),raw_Data[['wyId','label']],how='left',on='wyId')
        .set_index('wyId'))

In [5]:
data.shape

(1414, 53)

In [6]:
data.loc[2499738]

dateutc                             2018-03-31 14:00:00
competitionId                                       364
seasonId                                         181150
roundId                                         4405654
winner                                             1633
teamId_home                                        1633
score_home                                            3
scoreHT_home                                          3
teamId_away                                        1619
score_away                                            0
scoreHT_away                                          0
HS                                                   13
AS                                                    9
HST                                                   5
AST                                                   0
HC                                                    5
AC                                                    9
HF                                              

In [7]:
data.loc[[2501028, 2500919, 2576085, 2565922, 2500897, 2500016, 2576077, 2500047, 2565856,
 2516902, 2499874, 2499882, 2500940, 2517036, 2516833, 2516870, 2500082, 2576215,
 2499884, 2516891]].label

wyId
2501028                           Lille - Metz, 3 - 1
2500919             Olympique Marseille - Metz, 6 - 3
2576085                   Sampdoria - Juventus, 3 - 2
2565922              Barcelona - Real Sociedad, 1 - 0
2500897             Caen - Olympique Marseille, 0 - 2
2500016           Stoke City - Manchester City, 0 - 2
2576077                        Lazio - Udinese, 3 - 0
2500047                      Watford - Burnley, 1 - 2
2565856               Barcelona - Legan\u00e9s, 3 - 1
2516902          Hoffenheim - Bayer Leverkusen, 1 - 4
2499874      Newcastle United - Leicester City, 2 - 3
2499882              West Ham United - Arsenal, 0 - 0
2500940         Olympique Marseille - Bordeaux, 1 - 0
2517036        Bayern M\u00fcnchen - Stuttgart, 1 - 4
2516833         Borussia M'gladbach - Mainz 05, 1 - 1
2516870                  Augsburg - Hertha BSC, 1 - 1
2500082                    Chelsea - Liverpool, 1 - 0
2576215                      Cagliari - Napoli, 0 - 5
2499884    Manchester U

In [8]:
data.columns

Index(['dateutc', 'competitionId', 'seasonId', 'roundId', 'winner',
       'teamId_home', 'score_home', 'scoreHT_home', 'teamId_away',
       'score_away', 'scoreHT_away', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC',
       'HF', 'AF', 'B365H', 'B365D', 'B365A', 'y0_sh_H', 'x0_sh_H',
       'acc_avg_sh_H', 'goal_avg_sh_H', 'y0_sh_A', 'x0_sh_A', 'acc_avg_sh_A',
       'goal_avg_sh_A', 'mins4_H', 'mins4_A', 'shots_11H', 'shots_11A',
       'shots_acc_11H', 'shots_acc_11A', 'goals_H', 'goals_A', 'goals_ratio_H',
       'goals_ratio_A', 'passes_11H', 'passes_11A', 'passes_acc_11H',
       'passes_acc_11A', 'keyPasses_H', 'keyPasses_A', 'ataque_h', 'defensa_h',
       'ataque_a', 'defensa_a', 'ataquedefensa_H', 'ataquedefensa_A', 'label'],
      dtype='object')

In [ ]:
data.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')

In [10]:
results = pd.read_csv(path_graphs+'plot_results//train_preds_df.csv',sep=';')
results

FileNotFoundError: [Errno 2] No such file or directory: 'F:/TFG/graphs/plot_results/train_preds_df.csv'

We only want the data from the last 4 matches as training data.

In [11]:
labels = []

for match in data.itertuples():
    if match.winner == 0: labels.append(0)
    elif match.winner == match.teamId_home: labels.append(1)
    else: labels.append(2)

# data['res'] = target
label_set = pd.DataFrame({'Labels':labels}, index=data.index)
label_set.head(3)

,Labels
wyId,
2499738,1
2499799,1
2499800,2


In [12]:
training_features = ['mins4_H', 'mins4_A', 'shots_11H', 'shots_11A',
       'shots_acc_11H', 'shots_acc_11A', 'goals_H', 'goals_A', 'goals_ratio_H',
       'goals_ratio_A', 'passes_11H', 'passes_11A', 'passes_acc_11H',
       'passes_acc_11A', 'keyPasses_H', 'keyPasses_A', 'ataque_h', 'defensa_h',
       'ataque_a', 'defensa_a', 'ataquedefensa_H', 'ataquedefensa_A']

In [13]:
X_df = data[training_features]
# X_df.to_csv(path_train+'training_features_DF.csv',sep=';')
X_df.head(5)

,mins4_H,mins4_A,shots_11H,shots_11A,shots_acc_11H,shots_acc_11A,goals_H,goals_A,goals_ratio_H,goals_ratio_A,...,passes_acc_11H,passes_acc_11A,keyPasses_H,keyPasses_A,ataque_h,defensa_h,ataque_a,defensa_a,ataquedefensa_H,ataquedefensa_A
wyId,,,,,,,,,,,,,,,,,,,,,
2499738,60.818182,56.840909,28,18,0.321429,0.444444,1,3,0.250000,0.500000,...,0.757778,0.782137,5,6,0.76,1.72,0.96,2.24,0.973546,0.817933
2499799,54.500000,72.454545,35,28,0.400000,0.321429,4,5,1.000000,0.625000,...,0.851005,0.843464,4,8,0.56,0.84,2.00,1.20,1.316667,0.810784
2499800,63.295455,72.000000,37,29,0.162162,0.344828,1,1,0.333333,1.000000,...,0.836437,0.858100,16,7,0.60,1.20,0.48,1.72,0.983333,1.399187
2499801,42.000000,54.272727,28,22,0.392857,0.409091,2,4,1.000000,0.571429,...,0.785348,0.842152,8,3,0.36,2.16,1.04,0.32,0.689456,1.360256
2499802,63.659091,51.477273,47,25,0.510638,0.480000,10,4,0.555556,0.666667,...,0.904959,0.763764,15,2,5.48,1.08,1.24,0.64,1.935507,0.921212


In [14]:
y = label_set
X = data[training_features].dropna()

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=1100, random_state=1
)

In [ ]:
X_train.head(5)

In [37]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1100, 22), (1100, 1), (314, 22), (314, 1))

In [38]:
X_train.to_csv(path_train+'X_wys_train.csv',sep=';')
y_train.to_csv(path_train+'y_wys_train.csv',sep=';')
X_test.to_csv(path_train+'X_wys_test.csv',sep=';')
y_test.to_csv(path_train+'y_wys_test.csv',sep=';')

## Historical processing

In [3]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import re

In [4]:
df = pd.read_csv(path_rawdata+'historical_goals'+'.csv',sep=';',index_col='matchId').drop(columns='aux')

In [5]:
df.shape

(45383, 90)

In [6]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'season',
       'IdH', 'IdA', 'side_avg_1_home', 'Scored_1_home', 'Received_1_home',
       'points_1_home', 'side_avg_1_away', 'Scored_1_away', 'Received_1_away',
       'points_1_away', 'side_avg_2_home', 'Scored_2_home', 'Received_2_home',
       'points_2_home', 'side_avg_2_away', 'Scored_2_away', 'Received_2_away',
       'points_2_away', 'side_avg_3_home', 'Scored_3_home', 'Received_3_home',
       'points_3_home', 'side_avg_3_away', 'Scored_3_away', 'Received_3_away',
       'points_3_away', 'side_avg_4_home', 'Scored_4_home', 'Received_4_home',
       'points_4_home', 'side_avg_4_away', 'Scored_4_away', 'Received_4_away',
       'points_4_away', 'side_avg_5_home', 'Scored_5_home', 'Received_5_home',
       'points_5_home', 'side_avg_5_away', 'Scored_5_away', 'Received_5_away',
       'points_5_away', 'side_avg_6_home', 'Scored_6_home', 'Received_6_home',
       'points_6_home', 'side_avg_6_away', 'Scored_6_aw

In [20]:
class FootballMatchesDataset(Dataset):
    def __init__(self,file,dataset='',drop=[]):

        str_drop = ''
        for d in drop:
            str_drop += '(_' + str(d) + ')|'
        if str_drop=='': str_drop='fhvcgh'

        pattern = re.compile(r'^((?!' + str_drop[:-1] + r').|(_home)|(_away))+$')

        # partidos para usar en entrenamiento y test (mismo conjunto en todos los datasets)
        np.random.seed(0)
        permutation = (np.random.permutation(np.arange(50000))[:37500] if (file=='train') 
                                else np.random.permutation(np.arange(50000))[37500:])
        

        df              = pd.read_csv(path_rawdata+dataset+'.csv',sep=';',index_col='matchId')
        self.features   = list(filter(pattern.match,df.columns[13:]))
        self.matches    = np.intersect1d(df[self.features].dropna().index.to_numpy(),permutation)
        # falta drop na
        self.data       = torch.tensor(df.loc[self.matches][self.features].values).float()
        self.labels     = F.one_hot((torch.tensor(df.FTR.loc[self.matches].values)),num_classes=3).float()

    def __len__(self):
        return self.data.shape[0]

    def shape(self):
        return self.data.shape

    def __getitem__(self,idx):
        sample  = self.data[idx]
        label   = self.labels[idx]
        match   = self.matches[idx]
        return sample, label, match

In [24]:
historical = FootballMatchesDataset('train','historical',drop=[2])

In [25]:
historical.data.shape

torch.Size([15272, 193])

In [26]:
historical[464]

(tensor([1.5000e+01, 5.0000e+00, 1.1000e+01, 2.0000e+00, 1.3000e+01, 1.8000e+01,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 2.0300e+02, 1.6130e+03,
         1.6230e+03, 1.0000e+00, 3.0000e+00, 5.0000e+00, 0.0000e+00, 1.0000e+01,
         7.0000e+00, 1.0000e+00, 9.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 3.0000e+00, 1.7000e+01, 9.0000e+00, 1.0000e+01,
         1.4000e+01, 2.0000e+00, 2.0000e+00, 2.0000e+00, 5.0000e+00, 7.0000e+00,
         1.0000e+00, 1.0333e+01, 7.3333e+00, 2.3333e+00, 9.3333e+00, 1.6667e+00,
         3.3333e-01, 1.0000e+00, 3.0000e+00, 0.0000e+00, 3.0000e+00, 1.2333e+01,
         5.6667e+00, 7.0000e+00, 1.5667e+01, 1.6667e+00, 1.3333e+00, 2.0000e+00,
         7.0000e+00, 7.0000e+00, 1.5000e+00, 1.2250e+01, 8.7500e+00, 3.5000e+00,
         1.1000e+01, 1.7500e+00, 7.5000e-01, 2.0000e+00, 4.0000e+00, 0.0000e+00,
         3.0000e+00, 1.2250e+01, 6.0000e+00, 7.5000e+00, 1.6750e+01, 2.0000e+00,
         1.0000e+00, 2.0000e